# Product Class

In [18]:
class Product:
    instances = []
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
        Product.instances.append(self)
        
    def update_quantity(self, new_quantity): 
        self.quantity = new_quantity
    def display_info(self):
        print("Product Name:{}\tPrice:{}\tQuantity:{}".format(self.name,self.price,self.quantity))
    @classmethod
    def getProducts(cls):
        return Product.instances

# Customer Class

In [19]:
class Customer:
    instances = []
    def __init__(self,name,email):
        self.name=name
        self.email=email
        #list of dictionaries: {product:quantity}
        self.order_history = []  
        Customer.instances.append(self)
    def add_order(self, order):
        """
        order : dictionary with {product:quantity,...}
        """
        self.order_history.append(order)
    def display_info(self):
        print(f"Customer Name: {self.name}\nEmail: {self.email}\n")
        ordercount=1
        for order in self.order_history:
            print(f"Order No.:{ordercount}\n")
            for key in order:
                print(f"\tProduct Name: {key.name}\tQuantity: {order[key]}\n")
            ordercount+=1;
            print("\n\n")
    @classmethod
    def getCustomers(cls):
        return Customer.instances

### Process Order for Cart

In [20]:
def process_order(product,quantity):
    if(product.quantity>=quantity):
        product.quantity-=quantity
        print(f"Order processed for {product.name}\tQuantity:{quantity}\tAmount:{product.price*quantity}")
        return True
    else:
        print(f"Order can't be processed for {product.name}")
        return False

# Cart class:

In [21]:
class Cart:
    def __init__(self):
        self.item={}

    def add_product(self, product_name):
        """
        product_name : string
        returns : current quantity of the corresponding product in cart
        """
        products = Product.getProducts(); #object list
        product = None
        for p in products:
            if(p.name == product_name):
                product = p
                break
        if product is None:
            print("Order can't be processed! Product doesn't exist!\n")
            return
        
        if product in self.item:
            if(product.quantity-self.item[product]>0): 
                self.item[product]+=1;
            else:
                print("Out of Stock!")
        else:
            if(product.quantity>0): 
                self.item[product]=1
            else:
                print("Out of Stock!")
            
    def check_out(self, customer_name):
        """
        customer_name : string
        """
        customers = Customer.getCustomers() #object list
        customer = None
        for c in customers:
            if(c.name == customer_name):
                customer = c
                break
        if customer is None:
            print("Order can't be processed! Customer doesn't exist\n")
            return
        
        processed_orders={}
        for product in self.item:
            if process_order(product ,self.item[product]):
                processed_orders[product] = self.item[product]
        if processed_orders:
            customer.add_order(processed_orders)
        self.item={}

# Save to File Functions

In [22]:
def save_to_file(filename, content):
    """
    content : list of objects : customers or products
    """
    if(filename == "products.txt"):
        products = content
        with open('products.txt', 'w') as f:
            for product in products:
                f.write(f"{product.name},{product.price},{product.quantity}\n")

    if(filename=="customers.txt"):
        customers = content
        with open('customers.txt', 'w') as f, open('order_history.txt','w') as f1:
            for customer in customers:
                f.write(f"{customer.name},{customer.email}\n")

                ######################################################
                #orderno,email,product_name,quantity,price per quantity
                ordercount=0
                for order in customer.order_history:
                    for key in order:
                        f1.write(f"{ordercount},{customer.email},")
                        f1.write(f"{key.name},{order[key]},{key.price}\n")
                    ordercount+=1;

# Load From File

In [23]:
def load_from_file(filename):
    if(filename == "products.txt"):
        with open("products.txt", "r") as f:
            lines = f.readlines()
            for line in lines:
                name, price, quantity = line.strip().split(',')
                p1 = Product(name,int(price),int(quantity))
    if(filename == "customers.txt"):
        with open("customers.txt", "r") as f,open ("order_history.txt", "r") as f1 :
            lines = f.readlines()
            for line in lines:
                name, email = line.strip().split(',')
                c1 = Customer(name,email)
                ##########################################
                orders = f1.readlines()
                for order in orders:
                    orderno, email, pname,quantity,price = order.strip().split(',')
                    if(email==c1.email):
                        index = int(orderno)
                        while len(c1.order_history)<=index:
                            c1.order_history.append({})
                            
                        products = Product.getProducts(); #object list
                        product = None
                        for p in products:
                            if(p.name == pname):
                                product = p
                                break
                                
                        c1.order_history[index][product]=int(quantity)
                

# Generate Sales Report

In [24]:
def generate_sales_report(customers):
    print("Sales Report:\n")
    revenue=0
    pq={} # product:quantity
    for c in customers:
        for order in c.order_history:    #order : dict   order_history: List of dictionaries
            for product, quantity in order.items():
                revenue+=(quantity*product.price)
                if product in pq:
                    pq[product]+=quantity
                else:
                    pq[product]=quantity
    #most popular product
    print(f"Revenue: {revenue}\n")
    print(f"Most Popular Products:\n")

    sorted_pq = list(sorted(pq.items(), key=lambda x: x[1], reverse=True))
    for key, value in sorted_pq[:3] :
        print(f"{key.name}:{value} sold\n")

    return revenue, sorted_pq[:3]
    

## Email Validation

In [38]:
import re
email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
def is_valid_email(email):
    return bool(re.match(email_pattern, email))

# INTERFACE

In [39]:
def main():
    load_from_file("products.txt")
    load_from_file("customers.txt")
    while True:
        print("\nE-Commerce Management System\n1. Add Product\n2. Add Customer\n3. Process Order\n4. Generate Sales Report\n5. Exit\n")
        try:
            choice = int(input("Enter your choice: "))
            if(choice==1):
                name = input("Enter Product Name: ")
                price = int(input("Enter Product Price: "))
                quantity = int(input("Enter Product Quantity: "))
                assert quantity>=0, "Quantity can't be negative\n"
                assert price>0, "Price must be positive\n"
                
                products = Product.getProducts();         #product list
                product = None
                for p in products:
                    if(p.name == name):
                        product = p
                        break
    
                if product is None:
                    p1 = Product(name, price, quantity)
                    print("Product added successfully.\n")
                    p1.display_info()
                    print('\n')
                else:
                    print("Product already exists!\n")
                    c = (input("Do you want to update price and quantity?(y/n): "))
                    if(c.lower()=='y' or c.lower()=='yes'):
                        product.quantity+=quantity
                        product.price=price
                        print(f"Quantity & Price updated!\nQuantity:{product.quantity}\tPrice:{product.price}\n")
                    else:
                        print("Quantity & Price not updated!\n")
                    
            elif(choice==2):
                name = input("Enter customer name: ")
                email = input("Enter customer email: ")
                assert is_valid_email(email), "Please input valid email!!"
                #check if customer is already there
                customers = Customer.getCustomers() #object list
                customer = None
                for c in customers:
                    if(c.email == email and c.name == name):
                        customer = c
                        break
    
                if customer is None:
                    c1 = Customer(name,email)
                    print("Customer added successfully!\n")
                    c1.display_info()
                else:
                    print("Customer already exists!\n")
    
            elif(choice==3):
                customer_name = input("Enter customer name: ")
                product_name = input("Enter product name: ")
                q = int(input("Enter Quantity: "))
                assert q>0, "quantity must be positive!\n"
                c1 = Cart()
                for i in range(q):
                    c1.add_product(product_name)
                c1.check_out(customer_name)
                print(f"Order processed successfully!\n")
                
            elif(choice==4):
                customers = Customer.getCustomers()
                generate_sales_report(customers)
            elif(choice==5):
                products = Product.getProducts();
                customers = Customer.getCustomers()
                save_to_file("products.txt",products)
                save_to_file("customers.txt",customers)
                print("Exiting...\n")
                break
            else:
                print("invalid choice!")
        except ValueError as e:
            print("input should be integer")
        except Exception as e:
            print( f"{type(e).__name__} : {e}")


In [40]:
main()


E-Commerce Management System
1. Add Product
2. Add Customer
3. Process Order
4. Generate Sales Report
5. Exit



Enter your choice:  2
Enter customer name:  som
Enter customer email:  som


AssertionError : Please input valid email!!

E-Commerce Management System
1. Add Product
2. Add Customer
3. Process Order
4. Generate Sales Report
5. Exit



Enter your choice:  2
Enter customer name:  som
Enter customer email:  som@gmail.com


Customer added successfully!

Customer Name: som
Email: som@gmail.com


E-Commerce Management System
1. Add Product
2. Add Customer
3. Process Order
4. Generate Sales Report
5. Exit



Enter your choice:  5


Exiting...



In [15]:
i=-1
try:
    if(i<=0):
        raise ValueError("i must be positive")
    
except ValueError as e:
    print(e)
print(i)

i must be positive
-1


In [16]:
i+='s'

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [17]:
try:
    choice = int(input("Enter your choice: "))


Enter your choice:  d


ValueError: invalid literal for int() with base 10: 'd'

In [24]:
customers = Customer.getCustomers()
products = Product.getProducts()

In [25]:
len(customers)

5

In [26]:
len(products)

9

In [12]:
Customer1 = Customer('Sayantika', 'sayantika84@gmail.com')
Customer2 = Customer('Somenath', 'somenathgarai@gmail.com')

In [20]:
Laptop1 = Product('Asus A1', 50000, 23)
Laptop2 = Product('Asus A2', 55000, 45)
Laptop3 = Product('Asus A3', 60000, 43)
Phone1 = Product('Mi A1', 6000, 2)
Laptop1.display_info()

Product Name:Asus A1	Price:50000	Quantity:23


In [14]:
Laptop1.update_quantity(25)
Laptop1.display_info()

Product Name:Asus A1	Price:50000	Quantity:25


In [15]:
Customer1.add_order({Laptop1 : 3, Laptop2 :4})
Customer1.add_order({Laptop1 : 1, Laptop2 :5})
Customer1.display_info()

Customer Name: Sayantika
Email: sayantika84@gmail.com

Order No.:1

	Product Name: Asus A1	Quantity: 3

	Product Name: Asus A2	Quantity: 4




Order No.:2

	Product Name: Asus A1	Quantity: 1

	Product Name: Asus A2	Quantity: 5






In [16]:
Customer2.display_info()

Customer Name: Somenath
Email: somenathgarai@gmail.com



## Error to Be Handled : print ("Product is not available")

In [71]:
Customer1.add_order({Laptop4 : 1, Laptop2 :5})

NameError: name 'Laptop4' is not defined

### handle product out of stock error while adding product, also product exists or not
#### add outofstockerror in else


In [74]:
c1 = Cart()
c1.add_product('Mi A1')
c1.add_product('Asus A1')
for key,value in c1.item.items():
    print(f"{key.name},{value}\n")

Mi A1,1

Asus A1,1



In [75]:
c1.checkout('Sayantika')
Customer1.display_info()

Order processed for Mi A1	Quantity:1	Amount:6000
Order processed for Asus A1	Quantity:1	Amount:50000
Customer Name: Sayantika
Email: sayantika84@gmail.com

Order No.:1

	Product Name: Asus A1	Quantity: 3

	Product Name: Asus A2	Quantity: 4




Order No.:2

	Product Name: Asus A1	Quantity: 1

	Product Name: Asus A2	Quantity: 5




Order No.:3

	Product Name: Mi A1	Quantity: 1

	Product Name: Asus A1	Quantity: 1






In [76]:
Phone1.quantity

1

### Get List of Customers and Products

In [79]:
save_to_file("products.txt",products)
save_to_file("customers.txt",customers)

In [17]:
customers = Customer.getCustomers()
products = Product.getProducts()

In [18]:
len(customers)

0

In [16]:
len(products)

0

In [19]:
load_from_file("products.txt")

In [20]:
len(customers)

2

In [21]:
for c in customers:
    print(c.name)

Sayantika
Somenath


In [22]:
len(products)

4

In [23]:
for p in products:
    print(p.name)

Asus A1
Asus A2
Asus A3
Mi A1


In [47]:
for c in customers:
    print(c.order_history)

[{<__main__.Product object at 0x000001AD084EA510>: 3, <__main__.Product object at 0x000001AD084EA540>: 4}, {<__main__.Product object at 0x000001AD084EA510>: 1, <__main__.Product object at 0x000001AD084EA540>: 5}, {<__main__.Product object at 0x000001AD084E8F80>: 1, <__main__.Product object at 0x000001AD084EA510>: 1}]
[{<__main__.Product object at 0x000001AD084EA510>: 3, <__main__.Product object at 0x000001AD084EA540>: 4}, {<__main__.Product object at 0x000001AD084EA510>: 1, <__main__.Product object at 0x000001AD084EA540>: 5}, {<__main__.Product object at 0x000001AD084E8F80>: 1, <__main__.Product object at 0x000001AD084EA510>: 1}]


In [48]:
for c in customers:
    c.display_info()

Customer Name: Sayantika
Email: sayantika84@gmail.com

Order No.:1

	Product Name: Asus A1	Quantity: 3

	Product Name: Asus A2	Quantity: 4




Order No.:2

	Product Name: Asus A1	Quantity: 1

	Product Name: Asus A2	Quantity: 5




Order No.:3

	Product Name: Mi A1	Quantity: 1

	Product Name: Asus A1	Quantity: 1




Customer Name: Somenath
Email: somenathgarai@gmail.com

Order No.:1

	Product Name: Asus A1	Quantity: 3

	Product Name: Asus A2	Quantity: 4




Order No.:2

	Product Name: Asus A1	Quantity: 1

	Product Name: Asus A2	Quantity: 5




Order No.:3

	Product Name: Mi A1	Quantity: 1

	Product Name: Asus A1	Quantity: 1






In [56]:
customers = Customer.getCustomers()
len(customers)

2

In [26]:
generate_sales_report(customers)

Revenue: 751000

Most Popular Products:

Asus A2:9 sold

Asus A1:5 sold

Mi A1:1 sold



(751000,
 [(<__main__.Product at 0x16261470290>, 9),
  (<__main__.Product at 0x16261470f50>, 5),
  (<__main__.Product at 0x16261470c20>, 1)])

In [21]:
user_process_order('Sayantika', "Asus A3",1)

In [23]:
Customer1.display_info()

Customer Name: Sayantika
Email: sayantika84@gmail.com

Order No.:1

	Product Name: Asus A1	Quantity: 3

	Product Name: Asus A2	Quantity: 4




Order No.:2

	Product Name: Asus A1	Quantity: 1

	Product Name: Asus A2	Quantity: 5




Order No.:3

	Product Name: Asus A3	Quantity: 1




